In [1]:
%matplotlib qt5
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as scyint
import pandas as pd
from mathphys import constants
from mathphys.functions import save_pickle, load_pickle
from mathphys.beam_optics import beam_rigidity

In [2]:
import touschek_pack.functions as tousfunc

In [3]:
import touschek_pack.Classes as tousclass

In [4]:
# eu preciso lembrar de depois acrescentar uma função que faça o trakcing nao tornando explicito a diferenciação entre desvios de energia 
# positivos e negativos para contabilizar de maneira correta a taxa de espalhamento touschek porque na biblioteca lifetime a taxa de espalhamento
# calculada esta implementada com base nos espalhamentos tanto negativos como positivos

In [5]:
# This function will probably will be in my repositories
# I dont know if I will use it again, but it seems to me that it could be uselfull in some point

def extract_delt(groups, deltas):
    c = 0
    big_list = []
    for lists in groups:
        lil_list = []
        for _ in lists:
            lil_list.append(c)
            c+=1
            
        big_list.append(lil_list)
    
    sep_deltas = []
    comp_l = []

    for iten in big_list:
        sep_deltas.append(deltas[iten])
        comp_l.append(len(iten))


    return sep_deltas, comp_l

In [6]:
acc = pymodels.si.create_accelerator()
acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(acc)

tousan = tousclass.Tous_analysis(acc)
spos = pyaccel.lattice.find_spos(acc, indices='closed')
s = [np.round([spos[30]], 2), np.round([spos[350]], 2)]
# s = 30

df = tousan.get_finally(s)

IndexError: invalid index to scalar variable.

In [11]:
c = 0
for iten in df[1]:
    if not iten:
        c+=1

c

22

In [7]:
df

([array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]),
  array([False, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False,  True, False, False,  True, False,
         False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False, False, False, False, False, False,
          True,  True,  True,  True, False, False, False, False, False,
         False,  True,  True,  True,  True,  True,  True, Fals

In [26]:
a = np.array([1,2,3,4,5,6,7])

b = np.array([1,2,3,4,9,12,34,56,75,96])


np.isin(a,b), np.isin(b,a)

(array([ True,  True,  True,  True, False, False, False]),
 array([ True,  True,  True,  True, False, False, False, False, False,
        False]))

In [ ]:
# l_ost, deltas = np.zeros(len(res)), np.zeros(len(res))
# for idx, iten in enumerate(res):
#     _, ellost, delta = iten
#     l_ost[idx] = ellost
#     deltas[idx] = delta


In [ ]:

ltime = pyaccel.lifetime.Lifetime(acc)
tous_rate = ltime.touschek_data['rate']
spos = pyaccel.lattice.find_spos(acc, indices='closed')

npt = int((spos[-1]-spos[0])/0.1)


scalc = np.linspace(spos[0], spos[-1], npt)

ltime = pyaccel.lifetime.Lifetime(acc)
tous_rate = ltime.touschek_data['rate']
rate_nom_lattice = np.interp(spos, scalc, tous_rate)

In [ ]:
spos = pyaccel.lattice.find_spos(acc, indices='closed')
i = np.argmin(np.abs(spos-np.round([spos[350]], 2)))

model = pymodels.si.create_accelerator()
model.cavity_on = True
model.radiation_on = True
model.vchamber_on = True


res = tousfunc.track_eletrons(tousan._deltas, tousan._nturns, i, model)


# para este caso específico a posição analisada é spos[1046], ou seja, o indice em que a simulação tracking começa é em 1046
# a para este valor teremos então diversas posições em que o eletron é perdido e a partir disso eu vou ter organizar meu pensamento

lostinds, deltas = np.zeros(len(res)), np.zeros(len(res))
for idx,iten in enumerate(res):
    tloss, ellost, delta = iten
    lostinds[idx] = ellost
    deltas[idx] = delta # alguns elétrons possuem desvio de energia abaixo da aceitancia e acabam não sendo perdidos

lostinds = np.intp(lostinds)

# deltas são os desvios de energia perdidos pelo tracking
lost_positions = spos[lostinds]
lost_positions = np.round(lost_positions, 2)

In [ ]:
np.unique(lost_positions)

In [ ]:

fact = 0.03
step = int((deltas[0]+deltas[-1])/fact)
itv_track = np.linspace(deltas[0], deltas[-1], step) # method learned by fac repositories

data = pd.DataFrame({'lost_pos_by_tracking': lost_positions}) # create the dataframe that is obtained by tracking
lost_pos_column = (data.groupby('lost_pos_by_tracking').groups).keys()
data = pd.DataFrame({'lost_pos_by_tracking':lost_pos_column}) # this step agroups the lost_positions

# scat_lost_df = pd.DataFrame(f'{s}':) # dataframe will contain the scattered positions and the lost positions after scattering

itv_delta = []
for current, next_iten in zip(itv_track, itv_track[1:]):
    data['{:.2f} % < delta < {:.2f} %'.format(current*1e2, next_iten*1e2)] = np.zeros(len(list(lost_pos_column))) # this step creates new columns in the dataframe and fill with zeros
    itv_delta.append((current, next_iten))
    # Next step must calculate each matrix element from the dataframe

var = list(data.index) 
# quando as duas variáveis são iguais isso acab resultando em um erro então estou colocando essa condição.
if var == lost_pos_column:
    pass
else:
    data = data.set_index('lost_pos_by_tracking')


for idx, lost_pos in enumerate(lost_positions): # essas duas estruturas de repetição são responsáveis por calcular 
    # o percentual dos eletrons que possuem um determinado desvio de energia e se perdem em um intervalo de desvio de energia específico
    delta = deltas[idx]
    lps = []
    for j, interval in enumerate(itv_delta):
        if j == 0:
            if interval[0]<= delta <= interval[1]:
                data.loc[lost_pos, '{:.2f} % < delta < {:.2f} %'.format(interval[0]*1e2, interval[1]*1e2)] += 1
        else:
            if interval[0]< delta <= interval[1]:
                data.loc[lost_pos, '{:.2f} % < delta < {:.2f} %'.format(interval[0]*1e2, interval[1]*1e2)] += 1

data = data / len(deltas)

npt = int((spos[-1]-spos[0])/0.1)


scalc = np.linspace(spos[0], spos[-1], npt)
rate_nom_lattice = np.interp(spos, scalc, tous_rate)

dic_res = {}
lost_pos_df = []
part_prob = []
for index, iten in data.iterrows():
    t_prob = 0
    for idx, m in enumerate(iten):
        t_prob += m
        if idx == iten.count()-1:
            part_prob.append(t_prob)
            lost_pos_df.append(index)

lost_pos_df = np.array(lost_pos_df)
part_prob = np.array(part_prob)

idx = np.argmin(np.abs(spos-np.round([spos[350]], 2)))

dic_res['lost_position'] = lost_pos_df
dic_res['scat. at {}'.format(s)] = part_prob * rate_nom_lattice[idx]

dataframe = pd.DataFrame(dic_res)

In [ ]:
meu_array = np.array(dataframe.index.tolist())
dic_res['lost_position'], np.isin(lost_pos_df[0],meu_array)

In [ ]:
pd.set_option('display.max_rows', None)

dataframe

In [ ]:
np.round(np.unique(spos[np.intp(l_ost)]),2), np.unique(lost_positions)

In [ ]:
# Criar um DataFrame de exemplo
data = {'A': [1, 2, 3], 'B': [4, 5, 6]}
df = pd.DataFrame(data)

# Adicionar uma nova linha ao DataFrame
novo_valor = 10
nova_linha = pd.Series({'A': novo_valor})

indice_nova_linha = len(df)  # Obtém o índice da nova linha

df.loc[indice_nova_linha] = nova_linha

df

In [ ]:

df_ltime = pd.DataFrame({'rate':rate_nom_lattice})

In [ ]:
ind = np.argmin(np.abs(spos-s))

rate_nom_lattice[ind], ind

In [ ]:
if data.values.sum() == 1:
    pass
else:
    data = data/len(deltas)

In [ ]:
ind = np.arange(len(df_ltime))
df_index = spos[ind]

def_df_index = np.round(df_index, 2)
def_df_index

In [ ]:
plt.figure()
plt.plot(spos, rate_nom_lattice*1e7)

In [ ]:
accep = pyaccel.optics.calc_touschek_energy_acceptance(acc)

scalc, daccp, daccn = tousfunc.get_scaccep(acc, accep)

In [ ]:
b1 = ltime.touschek_data['touschek_coeffs']['b1']
b2 = ltime.touschek_data['touschek_coeffs']['b2']



In [ ]:
touschek_scattering_rate = []
# for idx,iten in enumerate(daccp):
#     touschek_scattering_rate.append(ltime.f_integral_simps(iten, b1[idx], b2[idx]))


In [ ]:
test_acceptance = np.array([daccp[1],daccp[2], daccp[3]])



In [ ]:
def f_function_arg(kappa, kappam, b1_, b2_):

    tau = (np.tan(kappa)**2)[:, None]
    taum = (np.tan(kappam)**2)[None, :]
    ratio = tau/taum/(1+tau)
    return ratio, tau, taum


In [ ]:
import scipy.special as special

In [ ]:
def f_function_arg_or(kappa, kappam, b1_, b2_):

    tau = (np.tan(kappa)**2)[:, None]
    taum = (np.tan(kappam)**2)[None, :]
    ratio = tau/taum/(1+tau)
    arg = (2*tau+1)**2 * (ratio - 1)/tau
    arg += tau - np.sqrt(tau*taum*(1+tau))
    arg -= (2+1/(2*tau))*np.log(ratio)
    arg *= np.sqrt(1+tau)
    bessel = np.exp(-(b1_-b2_)*tau)*special.i0e(b2_*tau)
    return arg * bessel

In [ ]:
npts = 300
kappa = np.linspace(daccp, np.pi/2, npts+1)

var = f_function_arg_or(kappa, daccp,b1,b2)


In [ ]:
tau.squeeze().transpose()

In [ ]:
matrix = np.array([[1,2,3],
                   [0,2,3],
                   [1,2,3]])
matrix.shape

matrix[1,0]



In [ ]:
tau.squeeze().shape